In [4]:
import json
import requests
import datetime as dt

from IPython.display import Markdown, display, HTML

In [5]:
json_regions = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json"
with requests.get(json_regions) as req:
    data = json.loads(req.content.decode('utf-8-sig'))

In [6]:
print("FIRST ENTRY DATE: {}".format(
    data[0]["data"]
    )
)
print("LAST  ENTRY DATE: {}".format(
    data[-1]["data"]
    )
)
period = (
    dt.datetime.strptime(data[-1]["data"], "%Y-%m-%dT%H:%M:%S") -
    dt.datetime.strptime(data[0]["data"], "%Y-%m-%dT%H:%M:%S")
).days

print("COVERAGE: {} days".format(period))
print("CURRENT DATE IS: {}".format(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

FIRST ENTRY DATE: 2020-02-24T18:00:00
LAST  ENTRY DATE: 2020-04-15T17:00:00
COVERAGE: 50 days
CURRENT DATE IS: 2020-04-15 21:26:30


In [7]:
x = []         # giorni

TOTyC = {}     # casi totali CUMULATIVO
yC = {}        # variazione giornaliera casi totali
TOTyD = {}     # deceduti CUMULATIVO
TOTyR = {}     # dimessi/guariti CUMULATIVO
TOTyP = {}     # infetti ISTANTANEO
yP = {}        # variazione giornaliera infetti

TOTyPric = {}  # ricoverati ISTANTANEO
TOTyPint = {}  # intensiva ISTANTANEO
TOTyPosp = {}  # totale ospedalizzati ISTANTANEO
TOTyPiso = {}  # isolamento ISTANTANEO???

TOTytamp = {}  # tamponi CUMULATIVO

In [8]:
for entry in data:

    # x values
    date = dt.datetime.strptime(entry["data"], "%Y-%m-%dT%H:%M:%S")
    if date not in x:
        x.append(date)
        
    region = entry["denominazione_regione"]
    
    if region not in TOTyC:
        TOTyC.update({region: []})
        yC.update({region: []})
        TOTyD.update({region: []})
        TOTyR.update({region: []})
        TOTyP.update({region: []})
        yP.update({region: []})
        TOTyPric.update({region: []})
        TOTyPint.update({region: []})
        TOTyPosp.update({region: []})
        TOTyPiso.update({region: []})
        TOTytamp.update({region: []})

    TOTyC[region].append(entry["totale_casi"])  #
    yC[region].append(entry["nuovi_positivi"])  #
    
    TOTyD[region].append(entry["deceduti"])
    TOTyR[region].append(entry["dimessi_guariti"])
    
    TOTyP[region].append(entry["totale_positivi"])  #
    yP[region].append(entry["variazione_totale_positivi"])  #

    TOTyPric[region].append(entry["ricoverati_con_sintomi"])  #
    TOTyPint[region].append(entry["terapia_intensiva"])  #
    TOTyPosp[region].append(entry["totale_ospedalizzati"])  #
    TOTyPiso[region].append(entry["isolamento_domiciliare"])  #
    
    TOTytamp[region].append(entry["tamponi"])

In [59]:
css = """
<style>
th {
    background-color: #BBDEFB;
    position:sticky;
    top: 0;
}
</style>
"""

headers = """
<th>REGIONE</th>
<th>data</th>
<th>TOT. CASI</th>
<th>VAR. CASI</th>
<th>INFETTI</th>
<th>VAR. INF.</th>
<th>TOT. OSP.</th>
<th>RICOV.</th>
<th>INTENS.</th>
<th>ISOL.</th>
<th>GUARITI</th>
<th>DECEDUTI</th>
<th>TAMPONI</th>
"""

line = ""
for region in TOTyC:
    line += "<tr><td style='text-align:center;font-size:200%;' rowspan='{}'><b>{}</b></td></tr>".format(period + 3, "<br>".join(list(region.upper())))
    for i in range(len(TOTyC[region])):
        line += ("<tr>" + 
            "<td>{}</td>".format(x[i].strftime("%Y %m %d")) +
            "<td style='color:brown'><b>{}</b></td>".format(TOTyC[region][i]) +
            "<td style='color:brown'>{:+}</td>".format(yC[region][i]) +
            "<td style='color:blue'><b>{}</b></td>".format(TOTyP[region][i]) +
            "<td style='color:blue'>{:+}</td>".format(yP[region][i]) +
            "<td>{}</td>".format(TOTyPosp[region][i]) +
            "<td style='color:grey'>{}</td>".format(TOTyPric[region][i]) +
            "<td style='color:grey'>{}</td>".format(TOTyPint[region][i]) +
            "<td  style='color:magenta'>{}</td>".format(TOTyPiso[region][i]) +
            "<td style='color:green'><b>{}</b></td>".format(TOTyR[region][i]) +
            "<td style='color:red'><b>{}</b></td>".format(TOTyD[region][i]) +
            "<td><i>{}</i></td>".format(TOTytamp[region][i]) +
            "</tr>")

print("""
TOT. CASI = INFETTI + GUARITI + DECEDUTI
INFETTI = TOT. OSP. + ISOL.
TOT. OSP. = RICOV. + INTENS.

CUMULATIVI:
- TOT. CASI
- GUARITI
- DECEDUTI
- TAMPONI

ISTANTANEI:
- INFETTI (quindi anche i suoi componenti)
? ISOLAMENTI ? dovrebbe...
""")
        
display(HTML(
    css + 
    "<div style='display: inline-block;overflow: auto;height: 600px;'><table><tr>{}</tr>".format(headers)
    + line + "</table>"
    
))


TOT. CASI = INFETTI + GUARITI + DECEDUTI
INFETTI = TOT. OSP. + ISOL.
TOT. OSP. = RICOV. + INTENS.

CUMULATIVI:
- TOT. CASI
- GUARITI
- DECEDUTI
- TAMPONI

ISTANTANEI:
- INFETTI (quindi anche i suoi componenti)
? ISOLAMENTI ? dovrebbe...

